In [ ]:
import pymrio as mr
import pandas as pd
# import numpy as np
# import scipy as sp
import pickle
import os
# from pypardiso import spsolve
# from scipy.linalg import diagsvd
# import matplotlib.pyplot as plt
# import seaborn as sns

%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

# Importer EXIOBASE

Le téléchargement devrait être automatique avec la commande `mr.download_exiobase3` ci-dessous. Si ce n'est pas le cas, c'est toujours possible de télécharger les fichiers manuellement à partir de l'adresse 10.5281/zenodo.15689391.

In [ ]:
# Dossier contenant les fichiers EXIOBASE téléchargés
EXIOBASE_PATH = "D:/EXIOBASE/3.9.6/"

exiobase_dict = dict()

# Choisissez les années à importer
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

for year in years:

    if not os.path.isfile(EXIOBASE_PATH + f"IOT_{year}_ixi.zip"):

        exio_meta = mr.download_exiobase3(
            storage_folder=EXIOBASE_PATH,  # installation path
            system="ixi",  # pick "product-by-product" (pxp) or "industry-by-industry" (ixi)
            years=years,  # years to download
            doi="10.5281/zenodo.15689391",  # repository URL
        )


In [ ]:
for year in years:
    print(year)
    exiobase_dict[year] = mr.parse_exiobase3(path=EXIOBASE_PATH + f"IOT_{year}_ixi.zip")

In [ ]:
for year in years:
    print(year)
    exiobase_dict[year].calc_all()
    print(f"EXIOBASE system calculated for year {year}")

# Sauvegarde
Il est utile de sauvegarder les bases de données EXIOBASE déjà calculées en format `pickle`. Ce sera plus rapide de les réutiliser pour les analyses.

In [ ]:
import pickle

In [ ]:
# Add population data

population = pd.read_csv(
    EXIOBASE_PATH + "exiobase_population.txt", sep="\\t", index_col=0
)

for year, io in exiobase_dict.items():
    io.population = pd.DataFrame(
        columns=["population"],
        data=population.loc[year].values,
        index=population.columns,
    ).T
    io.calc_extensions()

    with open(f"../exiobase/exiobase369_ixi_{year}.pkl", "wb") as f:
        pickle.dump(exiobase_dict[year], f)